# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = pd.read_csv("../output_data/weather_data.csv")
file.dropna()
file.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,klyuchi,65,RU,1569424979,38,52.25,79.17,281.223,1.01
1,san pedro,0,AR,1569424979,31,-33.68,-59.66,301.480,1.79
2,miandrivazo,0,MG,1569424979,64,-19.53,45.46,297.923,4.15
3,cidreira,32,BR,1569424979,57,-30.17,-50.22,294.723,2.13
4,xiaolan,20,CN,1569424980,74,22.67,113.23,300.930,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lon"]]
humidity = file["Humidity"].astype(float)
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vac_file=file.loc[(file['Max Temp']>=200)&(file['Max Temp']<=285)]
wind_file=vac_file.loc[(vac_file['Wind Speed']<=10)]
cloud=wind_file.loc[(wind_file['Cloudiness']==0)]
cloud.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
22,beringovskiy,0,RU,1569424986,84,63.05,179.32,271.023,0.47
24,novyy urgal,0,RU,1569424987,79,51.07,132.56,273.023,1.11
33,vostok,0,RU,1569424989,77,46.45,135.83,277.323,1.68
68,saskylakh,0,RU,1569424995,91,71.97,114.09,274.223,4.28
88,ust-nera,0,RU,1569424998,69,64.57,143.24,266.923,2.11


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = cloud.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
#hotel_df = hotel_df.loc[:, ~hotel.columns.str.contains('^Unnamed')]
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}
# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lon = row["Lon"]
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    name_address = requests.get(base_url, params=params)
    # convert to json
    name_address = name_address.json()
    print(json.dumps(name_address, indent=4, sort_keys=True))
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 51.13899319999999,
                    "lng": 133.0437234
                },
                "viewport": {
                    "northeast": {
                        "lat": 51.14020422989272,
                        "lng": 133.0446965798927
                    },
                    "southwest": {
                        "lat": 51.13750457010728,
                        "lng": 133.0419969201073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "382f85e8c4247db8017321ef28a1867a508daf2e",
            "name": "Gostinitsa Ostrov",
            "opening_hours": {
                "open_now": true
            },
            "photos":

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 68.05040079999999,
                    "lng": 166.472936
                },
                "viewport": {
                    "northeast": {
                        "lat": 68.05172877989273,
                        "lng": 166.4749283798927
                    },
                    "southwest": {
                        "lat": 68.04902912010728,
                        "lng": 166.4722287201073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "5f8fd756518998775b1b4fc37769a2672c8a1907",
            "name": "Profil

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 62.78810480000001,
                    "lng": 22.8394014
                },
                "viewport": {
                    "northeast": {
                        "lat": 62.78942232989272,
                        "lng": 22.84087307989272
                    },
                    "southwest": {
                        "lat": 62.78672267010728,
                        "lng": 22.83817342010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "cd2405cf5fb9b7d5a6b97968ea1309e627283d99",
            "name": "Original Sokos Hotel Vaakuna, Sein\u00e4joki",
            "opening_hours": {
                "open_now": true
          

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAO9BvvjAZ2rz4g2Pm7GCPtLapTbDu6vg6Xs13tyg4tOSEtLgMczwDHETkrwDoW9c8J-l8oi35GzE96L5F-XwTmXXhXA5N0W27jvXNjHXStkBHDmg11oQx3osvFL-_pbOpyMrg8GphPArxBdWYFB2nO-YEQBeirPIzSn45RDYaRADv2BalcAwS5_tNzDs7U5WcViGgsnN12MLPKfkubeS4ErgrkK-iteQgnJAX0Is54l-SVXjVPhr3kUCktveyzdpnj40rfPP3goLN7cpAplQn2K9dHpcovVPHBhhI7bJeOCQ0v0MVXzRjH2D3ILfKrrsJLcI7B7OMjY29EG2h-D8WCJW15MXgeY8aiJ30jbYFhB9yyUN44z4rd4063XnOxnPkORoxypQ6rCvLRD9JeHG-f8SECaE_zKeQJbkVWkhaDtrWMoaFD5CLuOKbrZLK9X31u_CP7oPEBOW",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 36.059578,
                    "lng": 103.82086
                },
                "viewport": {
                    "northeast": {
                        "lat": 36.06080662989272,
                        "lng": 103.8222037798927
                    },
                    "southwest": {
                        "lat": 36.05810697010728,
                        "

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAKTvdI7_W1qf7NhOed5D2FBFiUUVbIXgDkkqKCaXbbOxKaw8g2Dr74f3GaqDIvHt-4FA1L4bu6Eotzz6EsuIzUCLN_Em88ArgUilB1PyHNWkbMmfK7MqSX4zy2aAiqKx4zgOV2ODe6eYxOZuaI5L9AXkEorIeAw4-z88A3yq37PtmhLAwnYGPkSl4hUbglixkwYlEY5ktxJZD6d9ngKGW1FkjxV1uLJZx8hgvnJEL0VixwwdMzv8ex-T6VWebyc8_XhhT1OrrygJuqSoFsXJaNdaRl929P0PFVmQIzoFbuCU9VAfqsdR6nh6gmprtXTi19FkdNe2YrteMFw7M9fiO6FMKQvv9JLKU49vPrkVijHCI_45nmx_bGL2o1QYrTGyxUHYwZ_ZEG1hkXgzjSed3yYSEC28fjgrFCuwEQ4el6B_iswaFNVQ1LtxSSFQdKpxG-hWLRveO_I8",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 34.1617163,
                    "lng": 77.58244479999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 34.16306417989272,
                        "lng": 77.58384077989272
         

...

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 60.73167400000001,
                    "lng": 77.592906
                },
                "viewport": {
                    "northeast": {
                        "lat": 60.73309902989273,
                        "lng": 77.59414952989273
                    },
                    "southwest": {
                        "lat": 60.73039937010729,
                        "lng": 77.59144987010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "f17430ecbe879e1d07ecc09c13f38c0af6b150b8",
            "name": "Sibiryachka",
            "photos": [
                {
                    "height": 1836,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/106657921008671904642/photos\"

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAALyx4zAIgZ4fIZLh51lyhWKmYKJIY-cdn5UU_aXm-s0ho8QvouOxdWerfR8G3n80iMdw_qWuANm4xZAMJKXjZr7efIUOp_ha6Gk8yaQ-xjFuLwVNNpOG5TIQ-0j0bWrcTTqY1OS8aoDTHuN4BhvpstzCMnJn20OKMOYUApfOvIV7NkHJIYxGVOsMu1UtUfLlzKm9ki4741Ecn09BMR3hcT1bCOnALDGX1HO1v4E9bgS3FeCmD47GHj6nNX5VGoSnCU-344Lr6p7ayuG7yAD2Bty0v1BpBGrWf8SG6NoH8M4ywMlKBIfOae3O2GXiaJOf1fFY0AEg6Lwih1-6KOQ8bC95Pqagqv4__qoOWHNC9OUtLMZEY_QpQLOlOjq66NA_7dw02X9qdg-adIlZ6YmyiA4SEA6M-4-nnqTjCZKNUp0yexYaFDHrl5hqmnEnevNOdz31bHh2r-GL",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -35.553661,
                    "lng": 138.624316
                },
                "viewport": {
                    "northeast": {
                        "lat": -35.55228812010728,
                        "lng": 138.6258242298927
                    },
                    "southwest": {
                        "lat": -35.55498777989272,
                     

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 61.2685003,
                    "lng": 128.1083416
                },
                "viewport": {
                    "northeast": {
                        "lat": 61.26985012989272,
                        "lng": 128.1096914298927
                    },
                    "southwest": {
                        "lat": 61.26715047010727,
                        "lng": 128.1069917701072
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "95ce5abb1556a23ce7c061a6b02c5782a2d4c3ac",
            "name": "Yelanka Tur",
            "photos": [
                {
                    "height": 3120,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/114650814170013270970/photos\">\u04

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAL-g6Ljq_bbMoUzfNyRRY4Y8wM5hZc6atGPwldhxqQ_fFGZINPAZS6e0ffvon4lv7t2wEideiBcMXXCf83HL6q9S0DI9HeGUFaFSV2NCLPQOwFHVHY3K2g0wJeXAKBGlHjYBNkltE1saDt7bV9pEv5PoEtoajdccu6yUFFW2u4Y1sU1O58FV3ZLsWz3zvh2Q-hS2YZOASikiXASNKzp9rDIl50Sn1rFLbOeT66amSArz-CX4yqsmMghuDKcFRDmk9HHHO-5coObCni6ErIQRpVcsrvLUVO0geEhmqssYXMpEp0H0U7AanZQZ9_PcGqfE07-MYl9ubu-Unioaucz5nHDzIn1bJfIPttUWzYIdIyviWvBcmvn3b6RYPycJhn90AxkP3RC11ugtfmRv_jkGBIMSEE1BMCRorukLuk4Ho8aS_9waFMrfP7xmO33Q3-B9-FusYN9weCM7",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 49.583178,
                    "lng": 128.47629
                },
                "viewport": {
                    "northeast": {
                        "lat": 49.58476212989272,
                        "lng": 128.4776238798927
                    },
                    "southwest": {
                        "lat": 49.58206247010727,
                        "

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed,Hotel Name
0,beringovskiy,0,RU,1569424986,84,63.05,179.32,271.023,0.47,
1,novyy urgal,0,RU,1569424987,79,51.07,132.56,273.023,1.11,Gostinitsa Ostrov
2,vostok,0,RU,1569424989,77,46.45,135.83,277.323,1.68,
3,saskylakh,0,RU,1569424995,91,71.97,114.09,274.223,4.28,
4,ust-nera,0,RU,1569424998,69,64.57,143.24,266.923,2.11,Gostevoy Dom
5,lavrentiya,0,RU,1569424999,89,65.58,-170.99,270.423,1.57,
6,cherskiy,0,RU,1569425000,81,68.75,161.30,269.223,1.50,
7,bilibino,0,RU,1569425001,93,68.06,166.44,263.223,1.32,Profilaktoriy Baes
8,shubarkuduk,0,KZ,1569425003,37,49.15,56.49,281.323,2.27,
9,vysokogornyy,0,RU,1569425009,93,50.10,139.13,272.123,1.11,Mini Hotel Uktur


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [9]:
# Add marker layer ontop of heat map
#fig_marker_layer = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))